# Stateless app, from zero to Docker to Kubernetes

I have decided that I want to learn Kubernetes. I'm going to start small by using the tools that I already know, then build my way in to Kubernetes.



I'm rebuilding my home lab, and this time I want to go full kubernetes. I'm tired of treating my stuff like pets, and have gotten tired of doing maintene.ce I've resisted Kubernetes for a long time, but I finally think I'm ready to dive in. I'm promising myself that I won't

I'm going to start by deploying `searxng` 
I picked `searxng` for this experiment because it is an extremely simple stateless service - the easiest kind to deploy.

I'm going to run `searxng`, a privacy focused Internet metasearch engine, with the Docker CLI, Docker Compose, `podman`, []

I have other, more complex stateful and statless services which I'd like to run through this process with as well.

- [Searxng]

## Docker
- [Docker](https://docs.docker.com/reference/cli/docker/) (requires [Docker Engine](https://docs.docker.com/engine/)
- [Podman](todo) (rootless, with [Podman](https://podman.io/docs/installation))

## Docker Compose
- [Docker Compose](https://docs.docker.com/compose/reference/) (rootless, with [podman-system-service](https://docs.podman.io/en/latest/markdown/podman-system-service.1.html))
- [Kompose](https://docs.docker.com/compose/reference/) (rootless, with [podman-system-service](https://docs.podman.io/en/latest/markdown/podman-system-service.1.html))

## Kubernetes
- [podman-kube-play](https://docs.podman.io/en/latest/markdown/podman-kube-play.1.html) (rootless, with [Podman](https://podman.io/docs/installation))
- [kind]() (rootless? untested with [podman-system-service](https://docs.podman.io/en/latest/markdown/podman-system-service.1.html)))
- [Kubernetes in production](todo)

- [argo/flux??](todo)

In [13]:
%%sh
docker run --rm --detach -p 8080:8080 docker.io/searxng/searxng

ad26fa5d4f980852d9d0f1e15dde44f48ad1841ad4534cab824c4531d112413f


In [22]:
%%sh
curl -I localhost:8080

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  5994    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 5994
Server-Timing: total;dur=2.771, render;dur=1.086
X-Content-Type-Options: nosniff
X-Download-Options: noopen
X-Robots-Tag: noindex, nofollow
Referrer-Policy: no-referrer
Connection: close



![searxng homepage displayed in my browser](searxng.png)

It worked! This isn't exactly rocket science (yet). Let's tear this down.

In [23]:
%%sh
docker ps

CONTAINER ID   IMAGE             COMMAND                  CREATED         STATUS         PORTS                                       NAMES
ad26fa5d4f98   searxng/searxng   "/sbin/tini -- /usr/…"   5 minutes ago   Up 5 minutes   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   unruffled_bassi


In [24]:
%%sh
docker stop unruffled_bassi

unruffled_bassi


# Docker Compose

We can make things much more declarative with Docker Compose. This is by far my most loved development tool right now. I'm planning on writing a blog post with all the tips and tricks I've learned about it over the years.

In [27]:
%%sh
cat > compose.yml << EOF
# check out composerize for ideas https://www.composerize.com/
services:
    searx:
        image: searxng/searxng
        ports:
            - 8080:8080
EOF

docker compose up --detach

 Network notebooks_default  Creating
 Network notebooks_default  Created
 Container notebooks-searx-1  Creating
 Container notebooks-searx-1  Created
 Container notebooks-searx-1  Starting
 Container notebooks-searx-1  Started


In [28]:
%%sh
docker compose ps

NAME                IMAGE             COMMAND                  SERVICE   CREATED          STATUS          PORTS
notebooks-searx-1   searxng/searxng   "/sbin/tini -- /usr/…"   searx     12 seconds ago   Up 11 seconds   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp


In [29]:
%%sh
curl -I localhost:8080

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  5994    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 5994
Server-Timing: total;dur=30.488, render;dur=28.368
X-Content-Type-Options: nosniff
X-Download-Options: noopen
X-Robots-Tag: noindex, nofollow
Referrer-Policy: no-referrer
Connection: close



Still works! Let's tear it down.

In [30]:
%%sh
docker compose down

 Container notebooks-searx-1  Stopping
 Container notebooks-searx-1  Stopped
 Container notebooks-searx-1  Removing
 Container notebooks-searx-1  Removed
 Network notebooks_default  Removing
 Network notebooks_default  Removed


#### Production

It's also worth nothing that If I wanted to deploy this app to production, I could!

All I would need is to set the [Docker context](https://docs.docker.com/engine/context/working-with-contexts/) (`$DOCKER_HOST=ssh://root@mymachine`) to a server with SSH access and Docker installed.

That's how I have built my homelab up until now. It seems simple - just spin up a VPS with your favorite OS, add your SSH key, and [`apt install docker.io`](https://docs.docker.com/engine/install/).

It's easy until your machine runs out of disk space, or you want to 

There are many mitigations, but I don't have time to build it myself. That's what it feels like I've been doing for the past few years - instead of running my stuff I find myself with broken machines and the fear of having to rebuild a machine from scratch if it goes down.

It isn't automated. Continuous Deployment is something I have wanted for a long time, but `docker compose up --detach` doesn't cut it. This approach is fragile and just doesn't scale I guess.

Docker Swarm seems better, but I think at that point all signs lead to kubernetes.

But idk docker compose is suuccchh a nice dev interface - it is the perfect one
Kubernetes is good for deployment, but not the other way around - it is a pain to define if you just want your app.

until you have to deal with compute and networking and storage all manually. I'm hoping Kubernetes can handle everything for me, so I can get back to running my stuff.

## Kubernetes

Here is where we switch from Docker to Kubernetes. The most simple I have found to run Kubernetes is with `podman-kube-play`. First, we will need to generate some yaml.

### podman-kube-play

In [3]:
%%sh
cat > pod.yaml << EOF
apiVersion: v1
kind: Pod
metadata:
    name: searxng
spec:
    containers:
      - name: searxng
        image: searxng/searxng
        ports:
        - containerPort: 8080
          hostPort: 8080
EOF

podman kube play pod.yaml

Pod:
dafcb2ec196c95ace1511fd831ee31326ed6be053b105b70a720e8ac24c0bb70
Container:
30526150790c26760591cb85b655c0c344a2f1755594d4bf24cb6c4fb3822386



In [4]:
%%sh
curl -I localhost:8080

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  5994    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 5994
Server-Timing: total;dur=29.385, render;dur=26.854
X-Content-Type-Options: nosniff
X-Download-Options: noopen
X-Robots-Tag: noindex, nofollow
Referrer-Policy: no-referrer
Connection: close



It works! We defined a [pod](https://kubernetes.io/docs/concepts/workloads/pods/) and ran it with Podman! Let's check out how this works internally.

In [5]:
%%sh
podman pod ps

POD ID        NAME        STATUS      CREATED        INFRA ID      # OF CONTAINERS
dafcb2ec196c  searxng     Running     7 minutes ago  4a44fd2570c8  2


In [10]:
%%sh
podman pod top searxng

USER        PID         PPID        %CPU        ELAPSED          TTY         TIME        COMMAND
root        1           0           0.000       8m54.340141932s  ?           0s          /sbin/tini -- /usr/local/searxng/dockerfiles/docker-entrypoint.sh 
searxng     2           1           0.000       8m54.34027952s   ?           0s          uWSGI master 
searxng     10          2           0.187       8m54.340335183s  ?           1s          uWSGI worker 1 
searxng     13          2           0.187       8m54.340405723s  ?           1s          uWSGI worker 2 
searxng     16          2           0.187       8m54.340456777s  ?           1s          uWSGI worker 3 
searxng     22          2           0.187       8m54.340504967s  ?           1s          uWSGI worker 4 
searxng     26          2           0.187       8m54.340555672s  ?           1s          uWSGI worker 5 
searxng     29          2           0.187       8m54.340604701s  ?           1s          uWSGI worker 6 
searxng     36

Podman pods are really cool. So is `podman-play-kube`. I'd like to do some more writing about them soon.

todo Learn more about pods and how they are different from Docker Compose here.

Let's clean up and move on.

In [11]:
%%sh
podman play kube pod.yaml --down

Pods stopped:
dafcb2ec196c95ace1511fd831ee31326ed6be053b105b70a720e8ac24c0bb70
Pods removed:
dafcb2ec196c95ace1511fd831ee31326ed6be053b105b70a720e8ac24c0bb70
Secrets removed:
Volumes removed:


## Kubernetes

Note that I think Docker Desktop supports this without `kind`, so that might be good to check out.

### `kind`

[kind](https://github.com/kubernetes-sigs/kind) is a neat project which creates an entire Kubernetes cluster in Docker. Let's see what that means:

In [21]:
%%sh
# All we need is Docker (or Podman! check out https://kind.sigs.k8s.io/docs/user/rootless/)
docker ps

# Let's make our cluster!
kind create cluster

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Creating cluster "kind" ...
 • Ensuring node image (kindest/node:v1.30.0) 🖼  ...
 ✓ Ensuring node image (kindest/node:v1.30.0) 🖼
 • Preparing nodes 📦   ...
 ✓ Preparing nodes 📦 
 • Writing configuration 📜  ...
 ✓ Writing configuration 📜
 • Starting control-plane 🕹️  ...
 ✓ Starting control-plane 🕹️
 • Installing CNI 🔌  ...
 ✓ Installing CNI 🔌
 • Installing StorageClass 💾  ...
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Not sure what to do next? 😅  Check out https://kind.sigs.k8s.io/docs/user/quick-start/


In [28]:
%%sh
# woah, what did that do?
docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED          STATUS          PORTS                       NAMES
c368f28f3b63   kindest/node:v1.30.0   "/usr/local/bin/entr…"   32 minutes ago   Up 32 minutes   127.0.0.1:38171->6443/tcp   kind-control-plane


The docs say we can [interact with our cluster](https://kind.sigs.k8s.io/docs/user/quick-start/#interacting-with-your-cluster) with `kubectl`.

In [29]:
%%sh
kubectl cluster-info --context kind-kind

Kubernetes control plane is running at https://127.0.0.1:38171
CoreDNS is running at https://127.0.0.1:38171/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


Looks like we are Kuberneteseing!

I want to deploy that searxng pod, but this time on our Kubernetes cluster.

In [23]:
%%sh
kubectl apply -f pod.yaml

pod/searxng created


In [25]:
%%sh
kubectl get pods

NAME      READY   STATUS    RESTARTS   AGE
searxng   1/1     Running   0          19s


In [26]:
%%sh
curl -I localhost:8080

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8080 after 0 ms: Couldn't connect to server


CalledProcessError: Command 'b'curl -I localhost:8080\n'' returned non-zero exit status 7.

Looks like the pod is running, but I can't reach it from my host machine. Let's debug:

todo write blog post about kubernetes tooling

In [34]:
%%sh
kubectl describe pod searxng

Name:             searxng
Namespace:        default
Priority:         0
Service Account:  default
Node:             kind-control-plane/172.19.0.2
Start Time:       Sun, 23 Jun 2024 08:13:37 -0400
Labels:           <none>
Annotations:      <none>
Status:           Running
IP:               10.244.0.5
IPs:
  IP:  10.244.0.5
Containers:
  searxng:
    Container ID:   containerd://cadee59e9980ebccaf5426601cef84432917d25589a89a3d8a1dd906f2014b1f
    Image:          searxng/searxng
    Image ID:       docker.io/searxng/searxng@sha256:8409d05a16569230fb863e39aeb680132979b65ce5220f7ffcbf44017e8e4204
    Port:           8080/TCP
    Host Port:      8080/TCP
    State:          Running
      Started:      Sun, 23 Jun 2024 08:13:43 -0400
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from kube-api-access-hrmcl (ro)
Conditions:
  Type                        Status
  PodReadyToStartContainers   True 
  Initia

Looks like the pod is running, and the port configuration seems to be there. Maybe searxng is having some errors?

In [39]:
%%sh
# I could also try kubectl logs searxng to see the logs
kubectl exec searxng -- ps aux

PID   USER     TIME  COMMAND
    1 root      0:00 /sbin/tini -- /usr/local/searxng/dockerfiles/docker-entrypoint.sh
   13 searxng   0:00 {uwsgi} uWSGI master
   21 searxng   0:01 {Thread-19 (init} uWSGI worker 1
   22 searxng   0:01 {Thread-19 (init} uWSGI worker 2
   29 searxng   0:01 {Thread-19 (init} uWSGI worker 3
   32 searxng   0:01 {Thread-19 (init} uWSGI worker 4
   37 searxng   0:01 {Thread-19 (init} uWSGI worker 5
   42 searxng   0:01 {Thread-19 (init} uWSGI worker 6
   50 searxng   0:01 {Thread-19 (init} uWSGI worker 7
   55 searxng   0:01 {Thread-19 (init} uWSGI worker 8
   60 searxng   0:01 {Thread-19 (init} uWSGI worker 9
   66 searxng   0:01 {Thread-19 (init} uWSGI worker 10
   71 searxng   0:01 {Thread-19 (init} uWSGI worker 11
   76 searxng   0:01 {Thread-19 (init} uWSGI worker 12
   81 searxng   0:01 {Thread-19 (init} uWSGI worker 13
   86 searxng   0:01 {Thread-19 (init} uWSGI worker 14
   87 searxng   0:01 {Thread-19 (init} uWSGI worker 15
   94 searxng   0:01 {Thre

In [46]:
%%sh
kubectl exec searxng -- wget 127.0.0.1:8080

Connecting to 127.0.0.1:8080 (127.0.0.1:8080)
saving to 'index.html'
index.html           100% |********************************|  5994  0:00:00 ETA
'index.html' saved


Looks like `searxng` is running, but we just can't access it from outside the cluster. Per the [kind quick-start](https://kind.sigs.k8s.io/docs/user/quick-start/#mapping-ports-to-the-host-machine), we can map ports "from the nodes to the host machine with `extraPortMappings`". I'm not sure what that is, but I have a snippet that I can copy paste in. Let's try it.

#### Tooling aside

This is a great time to get to know some more Kubernetes tools. I'm going to start by adding shell completions to my favorite interactive shell, [fish](https://fishshell.com/). Kubernetes also provides `bash` and `zsh` completions [here](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/#enable-shell-autocompletion).

Also check out the [kubectl reference](https://kubernetes.io/docs/reference/kubectl/generated/). Also consider using a better user interface! Consider [k9s](https://github.com/derailed/k9s) for a TUI or maybe the [Kubernetes extension](https://code.visualstudio.com/docs/azure/kubernetes) for VSCode. There are many other tools which I'd like to write more about todo.

In [50]:
%%sh
# https://kind.sigs.k8s.io/docs/user/quick-start/#mapping-ports-to-the-host-machine
cat > kind-config.yaml << EOF
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
- role: control-plane
  extraPortMappings:
  - containerPort: 8080
    hostPort: 8080
EOF

kind delete cluster
kind create cluster --config kind-config.yaml
#kubectl apply -f cluster.yaml

Deleting cluster "kind" ...


Deleted nodes: ["kind-control-plane"]
Creating cluster "kind" ...
 • Ensuring node image (kindest/node:v1.30.0) 🖼  ...
 ✓ Ensuring node image (kindest/node:v1.30.0) 🖼
 • Preparing nodes 📦   ...
 ✓ Preparing nodes 📦 
 • Writing configuration 📜  ...
 ✓ Writing configuration 📜
 • Starting control-plane 🕹️  ...
 ✓ Starting control-plane 🕹️
 • Installing CNI 🔌  ...
 ✓ Installing CNI 🔌
 • Installing StorageClass 💾  ...
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! 👋


In [51]:
%%sh

kubectl get pods

No resources found in default namespace.


In [55]:
%%sh
kubectl apply -f pod.yaml

pod/searxng created


In [56]:
%%sh
curl -I localhost:8080

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


  0  5994    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 5994
Server-Timing: total;dur=43.089, render;dur=39.806
X-Content-Type-Options: nosniff
X-Download-Options: noopen
X-Robots-Tag: noindex, nofollow
Referrer-Policy: no-referrer
Connection: close



It worked! I had to reconfigure `kind` to map the container port to my host machine. This required completely destroying the cluster and recreating it from scratch.

Let's clean up.

In [58]:
%%sh
kind delete cluster
# all gone!
docker ps

Deleting cluster "kind" ...
Deleted nodes: ["kind-control-plane"]


CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
